# Distribution plot for each rule

In [9]:
import os
import json
from tqdm import tqdm
from collections import defaultdict as ddict
order_l = ["conclusion_first","premise_first"]
available_models = ["gpt"]
concat_file_name = "concat.json"
max_node_file_name = "max_node_len.json"
tail_head_list_template = ["all_tail_{rule_index}_nodelen_max","all_head_{rule_index}_nodelen_max"]
all_concat_data = ddict(lambda : ddict(lambda : ddict( dict)))


In [ ]:
path_search = "path_to_likelihood_results_of_link"
path_baseline_gpt35 = "path_to_likelihood_results_of_chatgpt"
path_baseline_gpt4 = "path_to_likelihood_results_of_gpt4"

is_search_or_baseline_list = ["search_rule","baseline_gpt3.5_rule","baseline_gpt4_rule"]
is_search_or_baseline_list = ["LINK","ChatGPT","GPT4"]
search_or_baseline_path = [path_search,path_baseline_gpt35,path_baseline_gpt4]

all_rule_keys = {
    # "conclusion_first": [0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18],
    # "premise_first": [2, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 69, 70, 71, 72, 73, 74, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 130, 131, 132, 133, 134, 135]
    "conclusion_first": [],
    "premise_first": [65, 66, 67, 68, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200]
}


for index,path in enumerate(search_or_baseline_path):

    is_search_or_baseline = is_search_or_baseline_list[index]

    if 1:
        for order in order_l:
            concat_file = os.path.join(os.path.join(path,order),concat_file_name)
            max_node_file = os.path.join(os.path.join(path,order),max_node_file_name)
            with open(max_node_file) as f:
                rule_keys = list(json.load(f).keys())
            with open(concat_file) as f:
                concat_data = json.load(f)
            rule_keys = [f"rule{id}" for id in all_rule_keys[order]]
            for rule_key in rule_keys:
                try:
                    tail_head_list = [_.format(rule_index = rule_key) for _ in tail_head_list_template]
                    all_concat_data[is_search_or_baseline][rule_key]["tail"][available_models[0]] = concat_data[available_models[0]][tail_head_list[0]]
                    all_concat_data[is_search_or_baseline][rule_key]["head"][available_models[0]] = concat_data[available_models[0]][tail_head_list[1]]
                except:
                    print(path, order, rule_key)

In [11]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import numpy as np

In [12]:
for rule in tqdm(all_concat_data["LINK"]):
    colors = ['blue', 'green', 'red']
    # 给定的三组列表数据
    list1 = all_concat_data["LINK"][rule]["tail"]["gpt"]
    list2 = all_concat_data["ChatGPT"][rule]["tail"]["gpt"]
    list3 = all_concat_data["GPT4"][rule]["tail"]["gpt"]
    hist_all = list1 + list2 + list3

    xmin_tail = np.min([np.min(arr) for arr in hist_all]) - 0
    xmax_tail = np.max([np.max(arr) for arr in hist_all]) + 0


    # ************************************************

    list4 = all_concat_data["LINK"][rule]["head"]["gpt"]
    list5 = all_concat_data["ChatGPT"][rule]["head"]["gpt"]
    list6 = all_concat_data["GPT4"][rule]["head"]["gpt"]
    hist_all = list4 + list5 + list6

    xmin_head = np.min([np.min(arr) for arr in hist_all]) - 0
    xmax_head = np.max([np.max(arr) for arr in hist_all]) + 0

    x_min = min(xmin_tail,xmin_head)
    x_max = max(xmax_tail,xmax_head)


    # ************************************************

    fig = make_subplots(rows=2, cols=1, subplot_titles=("Subplot 1", "Subplot 2"))


    # 第一个子图的三个直方图
    trace1 = go.Histogram(x=list1, opacity=0.5, name="LINK",xbins=dict(size=0.02),marker=dict(color=colors[0]))
    trace2 = go.Histogram(x=list2, opacity=0.5, name="ChatGPT",xbins=dict(size=0.02),marker=dict(color=colors[1]))
    trace3 = go.Histogram(x=list3, opacity=0.5, name="GPT4",xbins=dict(size=0.02),marker=dict(color=colors[2]))



    # 第二个子图的三个直方图
    trace4 = go.Histogram(x=list4, opacity=0.5, name="LINK",xbins=dict(size=0.02),marker=dict(color=colors[0]), showlegend=False)
    trace5 = go.Histogram(x=list5, opacity=0.5, name="ChatGPT",xbins=dict(size=0.02),marker=dict(color=colors[1]), showlegend=False)
    trace6 = go.Histogram(x=list6, opacity=0.5, name="GPT4",xbins=dict(size=0.02),marker=dict(color=colors[2]), showlegend=False)



    fig.add_trace(trace1,row=1, col=1)
    fig.add_trace(trace2,row=1, col=1)
    fig.add_trace(trace3,row=1, col=1)
    fig.add_trace(trace4,row=2, col=1)
    fig.add_trace(trace5,row=2, col=1)
    fig.add_trace(trace6,row=2, col=1)

    # 对齐x axes
    fig.update_xaxes(range=[x_min, x_max])



    # 更新布局
    fig.update_layout(
        barmode="overlay",  # 设置直方图模式为 overlay 以实现重叠效果
        title_text=f"{rule}",  # 设置总标题
        xaxis=dict(title="Value"),
        yaxis=dict(title="Frequency"),
    )

    fig.update_layout(
        annotations=[
            dict(text="Long-tail Distribution", x=0.5, xref="paper", y=1.0, yref="paper", showarrow=False, font_size=15),
            dict(text="Head Distribution", x=0.5, xref="paper", y=0.4, yref="paper", showarrow=False, font_size=15)
        ]
    )

    fig['layout']['yaxis2'].update(title_standoff=40) 


    fig.update_xaxes(title_text="Likelihood from text-davinci-003", row=1, col=1)
    fig.update_xaxes(title_text="Likelihood from text-davinci-003", row=2, col=1)
    # 显示图表
    # fig.show()
    fig.write_image(f"{rule}.pdf")


  0%|          | 0/108 [00:00<?, ?it/s]

100%|██████████| 108/108 [00:28<00:00,  3.81it/s]


## only use correct beams

In [13]:
only_use_correct_beams=True

In [23]:
order_l = ["conclusion_first","premise_first"]
available_models = ["gpt"]
concat_file_name = "concat.json"
max_node_file_name = "max_node_len.json"
tail_head_list_template = ["all_tail_{rule_index}_nodelen_max","all_head_{rule_index}_nodelen_max"]
all_concat_data = ddict(lambda : ddict(lambda : ddict( lambda : ddict(list))))

In [24]:
path_search = "path_to_likelihood_results_of_link"
path_baseline_gpt35 = "path_to_likelihood_results_of_chatgpt"
path_baseline_gpt4 = "path_to_likelihood_results_of_gpt4"

accept_beam_file = [
	"path_to_human_eval_results_of_link/accept_beams.json",
	"path_to_human_eval_results_of_chatgpt/accept_beams.json",
	"path_to_human_eval_results_of_gpt4/accept_beams.json"
	]
wrong_beam_file = [
	"path_to_human_eval_results_of_link/all_wrong_beams.json",
	"path_to_human_eval_results_of_link/all_wrong_beams.json",
	"path_to_human_eval_results_of_link/all_wrong_beams.json"
]
method_name_list = ["new_search", "new_baseline_chatgpt", "new_baseline_gpt4"]

is_search_or_baseline_list = ["search_rule","baseline_gpt3.5_rule","baseline_gpt4_rule"]
is_search_or_baseline_list = ["LINK","ChatGPT","GPT4"]
search_or_baseline_path = [path_search,path_baseline_gpt35,path_baseline_gpt4]

all_rule_keys = {
    # "conclusion_first": [15],
    "conclusion_first": [0],
    # "premise_first": [2, 29, 69, 88]
    "premise_first": [32, 46, 88, 112, 122]
}


for index,path in enumerate(search_or_baseline_path):

    is_search_or_baseline = is_search_or_baseline_list[index]

    if 1:
        for order in order_l:
            max_node_file = os.path.join(os.path.join(path,order),max_node_file_name)
            concat_file = os.path.join(os.path.join(path,order),concat_file_name)
            max_node_file = os.path.join(os.path.join(path,order),max_node_file_name)
            with open(max_node_file) as f:
                rule_keys = list(json.load(f).keys())
            with open(concat_file) as f:
                concat_data = json.load(f)
            with open(max_node_file) as f:
                node_max_len = json.load(f)
            rule_keys = [f"rule{id}" for id in all_rule_keys[order]]
            method_name = method_name_list[index]
            with open(accept_beam_file[index], "r") as f:
                accept_beam = json.load(f)[method_name]
            with open(wrong_beam_file[index], "r") as f:
                wrong_beam = json.load(f)[method_name]
            for rule_key in rule_keys:
                if 1:
                    tail_head_list = [_.format(rule_index = rule_key) for _ in tail_head_list_template]
                    for distribution in ["head", "longtail"]:
                        dist = "head" if distribution == "head" else "tail"
                        num = len(concat_data[available_models[0]][f"all_{dist}_{rule_key}_nodelen_max"])
                        for i in range(num):
                            if str(i) in accept_beam[rule_key][distribution]:
                                label = "correct"
                            elif str(i) in wrong_beam[rule_key][distribution]:
                                label = "incorrect"
                            else:
                                continue
                            if only_use_correct_beams:
                                if label != "correct":
                                    continue
                            all_concat_data[is_search_or_baseline][rule_key][dist][available_models[0]].append(concat_data[available_models[0]][f"{rule_key}_{dist}_{i}_{node_max_len[rule_key]}"][0])     
                # except Exception as e:
                #     print(path, order, rule_key, e)

In [36]:
for rule in tqdm(all_concat_data["LINK"]):
    colors = ['blue', 'green', 'red']
    # 给定的三组列表数据
    list1 = all_concat_data["LINK"][rule]["tail"]["gpt"]
    list2 = all_concat_data["ChatGPT"][rule]["tail"]["gpt"]
    list3 = all_concat_data["GPT4"][rule]["tail"]["gpt"]
    hist_all = list1 + list2 + list3

    xmin_tail = np.min([np.min(arr) for arr in hist_all]) - 0.1
    xmax_tail = np.max([np.max(arr) for arr in hist_all]) + 0.1


    # ************************************************

    list4 = all_concat_data["LINK"][rule]["head"]["gpt"]
    list5 = all_concat_data["ChatGPT"][rule]["head"]["gpt"]
    list6 = all_concat_data["GPT4"][rule]["head"]["gpt"]
    hist_all = list4 + list5 + list6

    xmin_head = np.min([np.min(arr) for arr in hist_all]) - 0.1
    xmax_head = np.max([np.max(arr) for arr in hist_all]) + 0.1

    x_min = min(xmin_tail,xmin_head)
    x_max = max(xmax_tail,xmax_head)


    # ************************************************

    fig = make_subplots(rows=2, cols=1, subplot_titles=("Long-tail Distribution", "Head Distribution"))


    # 第一个子图的三个直方图
    trace1 = go.Histogram(x=list1, opacity=0.5, name="LINK",xbins=dict(size=0.02),marker=dict(color=colors[0]))
    trace2 = go.Histogram(x=list2, opacity=0.5, name="ChatGPT",xbins=dict(size=0.02),marker=dict(color=colors[1]))
    trace3 = go.Histogram(x=list3, opacity=0.5, name="GPT4",xbins=dict(size=0.02),marker=dict(color=colors[2]))



    # 第二个子图的三个直方图
    trace4 = go.Histogram(x=list4, opacity=0.5, name="LINK",xbins=dict(size=0.02),marker=dict(color=colors[0]), showlegend=False)
    trace5 = go.Histogram(x=list5, opacity=0.5, name="ChatGPT",xbins=dict(size=0.02),marker=dict(color=colors[1]), showlegend=False)
    trace6 = go.Histogram(x=list6, opacity=0.5, name="GPT4",xbins=dict(size=0.02),marker=dict(color=colors[2]), showlegend=False)



    fig.add_trace(trace1,row=1, col=1)
    fig.add_trace(trace2,row=1, col=1)
    fig.add_trace(trace3,row=1, col=1)
    fig.add_trace(trace4,row=2, col=1)
    fig.add_trace(trace5,row=2, col=1)
    fig.add_trace(trace6,row=2, col=1)

    # 对齐x axes
    fig.update_xaxes(range=[x_min, x_max])

    for annotation in fig['layout']['annotations']: 
        annotation['font'] = dict(size=20)  # 设置字体大小

    fig.update_layout(
        title=f"Likelihood distribution for <b>{rule}</b> over <b>text-davinci-003</b>",
        barmode='overlay',
        title_font=dict(size=21)
    )

    fig.update_xaxes(row=1, col=1, title_text="Likelihood from text-davinci-003", title_font=dict(size=20))  # 用你的实际范围替换 xmin 和 xmax
    fig.update_xaxes(row=2, col=1, title_text="Likelihood from text-davinci-003", title_font=dict(size=20)) 
    fig.update_yaxes(row=1, col=1, range=[0, 15],title_text="# of statements", title_font=dict(size=17.5)) 
    fig.update_yaxes(row=2, col=1, range=[0, 15],title_text="# of statements", title_font=dict(size=17.5)) 

    # 显示图表
    fig.update_layout(
        legend_font=dict(size=15)         
    )

    import plotly.io as pio
    img_bytes = pio.to_image(fig, format="pdf", scale=7,width=800, height=600)
    with open(f"{rule}.pdf", "wb") as f:
        f.write(img_bytes)

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]
